In [ ]:
# Install required libraries
!pip install ultralytics
!pip install kagglehub
!pip install pyyaml
!pip install kagglehub

: 

In [ ]:
# Check GPU
!nvidia-smi

# Import and check pytorch with CUDA
import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")

In [5]:
# Code to run in new cell after model training

from google.colab import files
import tempfile
import os

def predict_uploaded_image(trainer):
    """
    Upload image and predict using trained model
    """
    print("Select image to upload...")

    # Upload image
    uploaded = files.upload()

    if not uploaded:
        print("No image uploaded!")
        return

    # Create temporary directory to save image
    with tempfile.TemporaryDirectory() as temp_dir:
        for filename, content in uploaded.items():
            # Save image to temp directory
            temp_path = os.path.join(temp_dir, filename)
            with open(temp_path, 'wb') as f:
                f.write(content)

            print(f"\nDự đoán cho ảnh: {filename}")
            # Thực hiện dự đoán
            trainer.predict_and_visualize(temp_path, conf=0.25)

In [ ]:
# Install Segment anything to focus to cell from uploaded image
!pip install git+https://github.com/facebookresearch/segment-anything.git

In [7]:
import torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
import numpy as np
import cv2
from google.colab import files
import tempfile
import os
import matplotlib.pyplot as plt
from PIL import Image

class BloodCellSegmentPredictor:
    def __init__(self, trainer, sam_checkpoint="sam_vit_h_4b8939.pth"):
        """
        Initialize predictor with SAM and YOLO model

        Args:
            trainer: Trained BloodCellTrainer object
            sam_checkpoint: Path to SAM weights
        """
        self.trainer = trainer

        # Load SAM model
        if not os.path.exists(sam_checkpoint):
            print("Downloading SAM checkpoint...")
            !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

        # Khởi tạo SAM
        device = "cuda" if torch.cuda.is_available() else "cpu"
        sam = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)
        sam.to(device=device)

        # Khởi tạo mask generator
        self.mask_generator = SamAutomaticMaskGenerator(
            model=sam,
            points_per_side=32,
            pred_iou_thresh=0.9,
            stability_score_thresh=0.96,
            crop_n_layers=1,
            crop_n_points_downscale_factor=2,
            min_mask_region_area=100,  # Adjust to filter out small regions
        )

    def extract_cell_regions(self, image):
        """
        Use SAM to segment cell regions

        Args:
            image: Numpy array BGR image

        Returns:
            List of (cropped_image, bbox)
        """
        # Convert to RGB for SAM
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Generate masks
        masks = self.mask_generator.generate(rgb_image)

        # Sort masks by area in descending order
        sorted_masks = sorted(masks, key=lambda x: x['area'], reverse=True)

        cell_regions = []
        for mask in sorted_masks:
            # Lấy bbox
            bbox = mask['bbox']  # x, y, w, h

            # Create crop with padding
            pad = 10
            x1 = max(0, bbox[0] - pad)
            y1 = max(0, bbox[1] - pad)
            x2 = min(image.shape[1], bbox[0] + bbox[2] + pad)
            y2 = min(image.shape[0], bbox[1] + bbox[3] + pad)

            # Crop image
            cropped = image[y1:y2, x1:x2]

            # Add to result list
            cell_regions.append({
                'crop': cropped,
                'bbox': (x1, y1, x2, y2),
                'mask': mask['segmentation']
            })

        return cell_regions

    def visualize_segments(self, image, cell_regions):
        """
        Display segmented regions
        """
        # Copy image for drawing
        vis_image = image.copy()

        # Draw bboxes and masks
        for idx, region in enumerate(cell_regions):
            x1, y1, x2, y2 = region['bbox']
            mask = region['mask']

            # Draw bbox
            cv2.rectangle(vis_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Draw mask with alpha blending
            colored_mask = np.zeros_like(image)
            colored_mask[mask] = [0, 0, 255]  # Red color for mask
            vis_image = cv2.addWeighted(vis_image, 0.7, colored_mask, 0.3, 0)

            # Add label
            cv2.putText(vis_image, f"Cell {idx+1}", (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        return vis_image

    def predict_regions(self, cell_regions, conf=0.25):
        """
        Predict cell type for each region
        """
        predictions = []
        for region in cell_regions:
            crop = region['crop']
            results = self.trainer.model.predict(crop, conf=conf)

            # Lấy thông tin dự đoán
            if len(results[0].boxes) > 0:
                for box in results[0].boxes:
                    pred = {
                        'bbox': region['bbox'],
                        'class': self.trainer.model.names[int(box.cls[0])],
                        'conf': float(box.conf[0])
                    }
                    predictions.append(pred)

        return predictions

    def process_image(self, image_path):
        """
        Process image: segment -> predict -> display
        """
        # Read image
        image = cv2.imread(image_path)
        if image is None:
            print("Image cannot be read!")
            return

        # Segment cell regions
        print("Detecting segment cell ...")
        cell_regions = self.extract_cell_regions(image)

        if not cell_regions:
            print("Not found!")
            return

        print(f"Found {len(cell_regions)} cell regions")

        # Display segments
        vis_image = self.visualize_segments(image, cell_regions)
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Segmented Cells')
        plt.show()

        # Predict each region
        print("\nPredicting cell type...")
        predictions = self.predict_regions(cell_regions)

        # Display results
        result_image = image.copy()
        for pred in predictions:
            x1, y1, x2, y2 = pred['bbox']
            label = f"{pred['class']} {pred['conf']:.2f}"

            # Draw bbox and label
            cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(result_image, label, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Detection Results')
        plt.show()

def predict_uploaded_image_with_sam(trainer):
    """
    Upload và dự đoán ảnh với SAM + YOLO
    """
    print("Select image to predict...")

    # Initialize predictor
    predictor = BloodCellSegmentPredictor(trainer)

    # Upload image
    uploaded = files.upload()

    if not uploaded:
        print("No image uploaded!")
        return

    # Process each uploaded image
    with tempfile.TemporaryDirectory() as temp_dir:
        for filename, content in uploaded.items():
            # Save image
            temp_path = os.path.join(temp_dir, filename)
            with open(temp_path, 'wb') as f:
                f.write(content)

            print(f"\nProcessing image: {filename}")
            predictor.process_image(temp_path)


In [ ]:
# Fix UTF-8 encoding
!apt-get update -qq
!apt-get install -qq locales
!locale-gen en_US.UTF-8
!update-locale LANG=en_US.UTF-8

import os
os.environ["LANG"] = "en_US.UTF-8"
os.environ["LC_ALL"] = "en_US.UTF-8"

# Install required libraries
!pip install ultralytics kagglehub matplotlib
!pip install git+https://github.com/facebookresearch/segment-anything.git

# Download SAM weights
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


In [9]:
import locale
print("Current locale:", locale.getpreferredencoding())

import torch
print("CUDA available:", torch.cuda.is_available())

# Check SAM weights
import os
print("SAM weights exist:", os.path.exists("sam_vit_h_4b8939.pth"))

Current locale: UTF-8
CUDA available: True
SAM weights exist: True


In [ ]:
import kagglehub
from ultralytics import YOLO
import yaml
from pathlib import Path
import shutil
import random
import cv2
import numpy as np
from IPython.display import display, Image
import matplotlib.pyplot as plt
import torch

def download_dataset():
    """
    Download dataset from Kaggle and return the path
    """
    try:
        path = kagglehub.dataset_download("quangnguynvnnn/bloodcell-yolo-format")
        print(f"Dataset downloaded successfully to: {path}")
        return Path(path)
    except Exception as e:
        print(f"Error downloading dataset: {e}")
        raise

class BloodCellTrainer:
    def __init__(self, model_name='yolov8n.pt'):
        """
        Initialize trainer for blood cell detection
        """
        self.data_dir = download_dataset()
        print(f"Using dataset from: {self.data_dir}")
        self.model = YOLO(model_name)

    def prepare_data_yaml(self, class_names):
        """
        Create data.yaml file for training

        Args:
            class_names: List of class names
        """
        data_yaml = {
            'path': str(self.data_dir.absolute()),
            'train': 'train/images',
            'val': 'valid/images',
            'test': 'test/images',
            'names': {i: name for i, name in enumerate(class_names)},
            'nc': len(class_names)
        }

        yaml_path = self.data_dir / 'data.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump(data_yaml, f)

        print(f"Created data.yaml at: {yaml_path}")
        return str(yaml_path)

    def split_dataset(self, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
        """
        Split dataset into train/val/test
        """
        # Create directories
        splits = ['train', 'valid', 'test']
        for split in splits:
            for subdir in ['images', 'labels']:
                split_dir = self.data_dir / split / subdir
                split_dir.mkdir(parents=True, exist_ok=True)

        # List all image files
        image_dir = self.data_dir / 'images'
        label_dir = self.data_dir / 'labels'

        image_files = list(image_dir.glob('*.jpg')) + list(image_dir.glob('*.png'))
        print(f"Found {len(image_files)} images in dataset")

        # Shuffle and split
        random.shuffle(image_files)
        n = len(image_files)
        train_end = int(n * train_ratio)
        val_end = int(n * (train_ratio + val_ratio))

        splits_files = {
            'train': image_files[:train_end],
            'valid': image_files[train_end:val_end],
            'test': image_files[val_end:]
        }

        # Copy files
        for split, files in splits_files.items():
            print(f"\nProcessing {split} split ({len(files)} images)...")
            for img_path in files:
                # Copy image
                shutil.copy2(img_path,
                           self.data_dir / split / 'images' / img_path.name)

                # Copy corresponding label file
                label_path = label_dir / (img_path.stem + '.txt')
                if label_path.exists():
                    shutil.copy2(label_path,
                               self.data_dir / split / 'labels' / label_path.name)

    def train(self, epochs=100, batch_size=16, img_size=640):
        """
        Training model
        """
        # Load data.yaml path
        yaml_path = str(self.data_dir / 'data.yaml')

        # Training configuration
        self.model.train(
            data=yaml_path,
            epochs=epochs,
            batch=batch_size,
            imgsz=img_size,
            device='0' if torch.cuda.is_available() else 'cpu',
            patience=50,
            save=True,
            project='blood_cell_detection',
            name='train',
            exist_ok=True,
            pretrained=True,
            optimizer='Adam',
            lr0=0.001,
            weight_decay=0.0005,
            warmup_epochs=3,
            close_mosaic=10,
            augment=True,
            cache=False,
            workers=8,
            save_period=10,
            verbose=True
        )

    def validate(self):
        """
        Validate model and display readable metrics
        """
        yaml_path = str(self.data_dir / 'data.yaml')
        results = self.model.val(data=yaml_path, split='val')

        # Format metrics into readable format
        metrics = {
            'mAP50': float(results.box.map50),
            'mAP50-95': float(results.box.map),
            'precision': float(results.box.mp),
            'recall': float(results.box.mr)
        }

        # Display metrics in table format
        print("\n=== Model Performance Metrics ===")
        print(f"{'Metric':<15} {'Value':>10}")
        print("-" * 30)
        for metric, value in metrics.items():
            print(f"{metric:<15} {value:>10.4f}")

        return metrics

    def predict_and_visualize(self, image_path, conf=0.25):
        """
        Predict and display results on image
        """
        # Perform prediction
        results = self.model.predict(image_path, conf=conf)

        # Get image with drawn boxes
        plotted_img = results[0].plot()

        # Chuyển từ BGR sang RGB
        plotted_img_rgb = cv2.cvtColor(plotted_img, cv2.COLOR_BGR2RGB)

        # Hiển thị ảnh sử dụng matplotlib
        plt.figure(figsize=(12, 8))
        plt.imshow(plotted_img_rgb)
        plt.axis('off')
        plt.title('Detection Results')
        plt.show()

        # In kết quả dự đoán
        print("\n=== Detection Results ===")
        boxes = results[0].boxes
        for box in boxes:
            # Get box coordinates
            x1, y1, x2, y2 = box.xyxy[0]
            # Get class and confidence
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            # Get class name from model
            class_name = self.model.names[cls]
            print(f"Class: {class_name:<10} Confidence: {conf:.4f}")

    def predict_folder(self, folder_path, num_images=5, conf=0.25):
        """
        Predict multiple images from a folder
        """
        # Get list of images
        image_files = list(Path(folder_path).glob('*.jpg')) + list(Path(folder_path).glob('*.png'))

        if not image_files:
            print(f"No images found in {folder_path}")
            return

        # Randomly select number of images to predict
        selected_images = random.sample(image_files, min(num_images, len(image_files)))

        # Predict and display each image
        for img_path in selected_images:
            print(f"\nProcessing image: {img_path.name}")
            self.predict_and_visualize(str(img_path), conf=conf)

global_trainer = None

def initialize_global_trainer(model_path=None):
    """Initialize global trainer with optional pre-trained model"""
    global global_trainer
    trainer = BloodCellTrainer(model_name='yolov8n.pt')
    
    # Prepare data.yaml
    class_names = ['rbc', 'wbc', 'platelets']
    trainer.prepare_data_yaml(class_names)
    
    # Split dataset
    trainer.split_dataset()
    
    # Load pre-trained weights if provided
    if model_path and os.path.exists(model_path):
        print(f"Loading pre-trained model from {model_path}")
        trainer.model = YOLO(model_path)
    
    global_trainer = trainer
    return trainer

def main(train_model=False):
    """
    Initialize and optionally train the model
    Args:
        train_model: Boolean to indicate if training should be performed
    """
    global global_trainer
    
    # Use existing trainer or create new one
    trainer = global_trainer if global_trainer is not None else initialize_global_trainer()
    
    if train_model:
        print("Starting model training...")
        trainer.train(epochs=10)
        print("\nValidating model...")
        metrics = trainer.validate()
        
        # Save trained model
        save_path = 'blood_cell_detection/train/weights/best.pt'
        if os.path.exists(save_path):
            print(f"\nSaved trained model to {save_path}")
            # Update global trainer with trained model
            initialize_global_trainer(save_path)
    
    return global_trainer

if __name__ == '__main__':
    main()

In [ ]:
# Prediction cell - can be run multiple times
print("=== Blood Cell Detection with Segment Anything ===")
print("Note: This cell can be run multiple times to process different images")

try:
    predict_uploaded_image_with_sam(global_trainer)
except NameError:
    print("Error: Trainer not initialized. Please run the initialization cell first.")
except Exception as e:
    print(f"Error during prediction: {e}")
    print("Please ensure the model is properly initialized before running predictions")